In [59]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization , Embedding , GlobalAveragePooling1D
from tensorflow.keras.layers import *

In [9]:
# 2 문자열을 벡터를 이용한 모델을 생성해서 값을 리턴받아보자
# 2-1 문장 데이터를 변수에 저장
input_array = np.array([[0,1,2,3,4],[5,1,2,3,6]])

# 2-2 랜덤 속성값 설정 (난수 똑같이나오게 지정)
np.random.seed(1)
tf.keras.utils.set_random_seed(2)

# 2-3 모델 생성하기
model = Sequential()
model.add(Embedding(7,2))
# model.add(GlobalAveragePooling1D()) # 평균리턴
model.add(LSTM(3, activation='tanh'))

model.compile('rmsprop' , 'mse')
model.fit(input_array)

output_array = model.predict(input_array)
print(output_array)
print(output_array.shape) # Embedding만하면 (2,5,2)
                          # GlobalAveragePooling1D하면 (2,2)

1/1 [==============================] - 0s 44ms/step
[[[ 0.02843983 -0.04518877]
  [ 0.01244213  0.02208966]
  [ 0.03567721  0.04178825]
  [ 0.04183492 -0.03163595]
  [-0.02534178 -0.01245432]]

 [[ 0.00433122 -0.04353247]
  [ 0.01244213  0.02208966]
  [ 0.03567721  0.04178825]
  [ 0.04183492 -0.03163595]
  [ 0.04732199  0.00539281]]]


(2, 5, 2)

In [ ]:
# 2 문자열을 벡터를 이용한 모델을 생성해서 값을 리턴받아보자
# 2-1 문장 데이터를 변수에 저장
input_array = np.array([[0,1,2,3,4],[5,1,2,3,6]])
# 2-2 랜덤 속성값 설정 (난수 똑같이나오게 지정)
np.random.seed(1)
tf.keras.utils.set_random_seed(2)

# 2-3 모델 생성하기
model = Sequential()
model.add(Embedding(7,2))
# model.add(GlobalAveragePooling1D()) # 평균리턴
model.compile('rmsprop' , 'mse')

output_array = model.predict(input_array)
print(output_array)
print(output_array.shape) # Embedding만하면 (2,5,2)
                          # GlobalAveragePooling1D하면 (2,2)

In [13]:
# 2 문자열을 벡터를 이용한 모델을 생성해서 값을 리턴받아보자
# 2-1 문장 데이터를 변수에 저장
input_array = np.array([[0,1,2,3,4],[5,1,2,3,6]]) # 텍스트를 토큰화
x_train = input_array.reshape(input_array.shape[0],input_array.shape[1],1)
y_train = np.array([7,8])
# 2-2 랜덤 속성값 설정 (난수 똑같이나오게 지정)
np.random.seed(1)
tf.keras.utils.set_random_seed(2)

# 2-3 모델 생성하기 : 출력은 시퀀스 다음에 어휘사전에 등장할 수 있는 단어의 확률
model = Sequential()
model.add(Embedding(7,2))
# model.add(GlobalAveragePooling1D()) # 평균리턴
model.add(LSTM(3, activation='tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile('rmsprop' , 'mse')
model.fit(x_train,y_train , batch_size=3, epochs=50, verbose=1)

output_array = model.predict(input_array)
print(output_array)
print(output_array.shape) # Embedding만하면 (2,5,2)
                          # GlobalAveragePooling1D하면 (2,2)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 49.2413
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 49.2074
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 49.1823
Epoch 4/50
1/1 [==============================] - 0s 5ms/step - loss: 49.1607
Epoch 5/50
1/1 [==============================] - 0s 5ms/step - loss: 49.1410
Epoch 6/50
1/1 [==============================] - 0s 7ms/step - loss: 49.1226
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 49.1050
Epoch 8/50
1/1 [==============================] - 0s 6ms/step - loss: 49.0881
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: 49.0717
Epoch 10/50
1/1 [==============================] - 0s 7ms/step - loss: 49.0556
Epoch 11/50
1/1 [==============================] - 0s 5ms/step - loss: 49.0397
Epoch 12/50
1/1 [==============================] - 0s 7ms/step - loss: 49.0241
Epoch 13/50
1/1 [==============================] - 0s 5ms/step

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [72]:
df = pd.read_csv('./my_data.csv')
no_of_lag = 3

#5-2 특징량 작성을 위한 쉬프트 함수를 만들어 결합
df_lag_X = pd.DataFrame()
for i in range(1,no_of_lag):
    df_lag_X['x_rag{}'.format(i)] = df['기온'].shift(i)
    
df_lag_y = pd.DataFrame()
for i in range(1,no_of_lag):
    df_lag_y['y_rag{}'.format(i)] = df['습도'].shift(i)
    
# 조인
df_temp = pd.concat([df['기온'] ,df_lag_X ] , axis=1)
df_humi = pd.concat([df['습도'] ,df_lag_y ] , axis=1)

df_temp.dropna(inplace=True)
df_humi.dropna(inplace=True)

# reshape로 일렬로 처리
temp = np.array(df_temp).reshape(-1,no_of_lag)
humi = np.array(df_humi).reshape(-1,no_of_lag)

# [batch , timesteps , feature]
input_format = []
_format1 = []
for j in range(0,len(temp)):
    for i in range(0,no_of_lag):
        _m = ( [temp[j][i] , humi[j][i]] )
        _format1.append(_m)
    input_format.append(_format1)
    _format1 = []

input_format = np.array(input_format)
y = pd.concat([df['날씨']],axis=1)
y = y[no_of_lag-1: ]

In [104]:
input_format.shape

(7, 3, 2)

In [73]:
train_x , test_x , train_y , test_y = train_test_split(input_format , y)
cb_e = tf.keras.callbacks.EarlyStopping(monitor='val_loss')
cb_save = tf.keras.callbacks.ModelCheckpoint(filepath='mycheck' , monitor='val_loss' , save_best_only=True)

In [74]:
train_y = np.array(train_y)
train_x = np.array(train_x)

In [78]:
# 5-9 모델작성
#LSTM 모델 작성 주의점 : 데이터의 순서가 새로운 시간의 데이터로 시작되기 때문에 (t , t-1 , t-2)

input01 = Input(shape = (train_x.shape[1] , train_x.shape[2]))
lstm = LSTM(16,go_backwards=True)(input01)
drop01 = Dropout(0.2)(lstm)
d1 = Dense(10)(drop01)
outputs = Dense(3)(d1)

model = Model(inputs = input01 , outputs = outputs)
model.compile('adam' , 'binary_crossentropy' , ['acc'])
model.fit(train_x , train_y , epochs = 5 , batch_size = 5 , validation_split=0.1, callbacks=[cb_e, cb_save])

Epoch 1/5
1/1 [==============================] - ETA: 0s - loss: 10.7642 - acc: 0.2500

INFO:tensorflow:Assets written to: mycheck\assets


INFO:tensorflow:Assets written to: mycheck\assets


1/1 [==============================] - 5s 5s/step - loss: 10.7642 - acc: 0.2500 - val_loss: 2.6479 - val_acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - ETA: 0s - loss: 7.7998 - acc: 0.2500

INFO:tensorflow:Assets written to: mycheck\assets


INFO:tensorflow:Assets written to: mycheck\assets


1/1 [==============================] - 3s 3s/step - loss: 7.7998 - acc: 0.2500 - val_loss: 1.7162 - val_acc: 0.0000e+00


In [79]:
pred_y = model.predict(test_x)
print(pred_y)

1/1 [==============================] - 0s 353ms/step
[[ 0.49463302 -0.03723762 -0.05681894]
 [ 0.48544335 -0.06318074 -0.1621387 ]]


In [88]:
from matplotlib import pyplot as plt
import seaborn as sns

In [81]:
df1 = pd.read_csv('./city_weather.csv')

In [85]:
df1['date'] = pd.to_datetime(df1['date'])
df1.set_index('date',inplace=True) # date를 인덱스로 변경
df1

,ave_temperature,ave_humidity,total_daylight_hours,total_precipitation,ave_wind_speed,ave_cloud_cover
date,,,,,,
2010-12-21,9.0,43,49.3,43.5,3.0,2.6
2010-12-28,6.5,43,48.7,0.0,2.4,4.1
2011-01-04,5.6,35,59.2,0.0,3.6,2.1
2011-01-11,4.0,36,50.8,0.0,2.7,3.8
2011-01-18,5.6,40,56.6,3.5,2.4,3.8
...,...,...,...,...,...,...
2020-11-17,17.0,65,46.9,0.0,3.2,5.1
2020-11-24,11.2,70,26.0,5.0,2.3,7.7
2020-12-01,8.6,77,30.5,11.5,2.0,5.6
